#### Requirements

In [1]:
import urllib
import urllib.request
from urllib.request import urlopen, Request
import urllib.parse
from urllib.error import HTTPError
from urllib.error import URLError
from http.cookiejar import CookieJar
from bs4 import BeautifulSoup
import lxml
import cchardet
import json, os
import pandas as pd
from fpdf import FPDF


#### Request

In [3]:
base_url = "https://api.elsevier.com/content/search/scopus?"

apiKey = "a981e389772e23df3d045cc51e2bcfc6"

query = "TITLE-ABS-KEY ({multiple sclerosis} AND {corpus callosum})"

view = "STANDARD"


try:
    url = base_url + "query=" + urllib.parse.quote(query) + "&apiKey=" + apiKey + "&view=" + view
    fetch_results = json.dumps(json.loads(urlopen(url).read()), indent=4)
    with open("scopus_search_results0.json", "w") as output:
        output.write(fetch_results)
    with open("scopus_search_results0.json", "r") as search_results:
        search_results = json.load(search_results)
    total_results = int(search_results["search-results"]["opensearch:totalResults"])

    start = 26
    int(total_results/(start-1))
    
    total_files = int(total_results/(start-1))
    counter = 1
    while counter <= total_files:
        url = base_url + "query=" + urllib.parse.quote(query) + "&apiKey=" + apiKey + "&view=" + view + "&start=" + str(start)
        fetch_results = json.dumps(json.loads(urlopen(url).read()), indent=4)
        with open(f"scopus_search_results{counter}.json", "w") as output:
            output.write(fetch_results)
        if start+25 < total_results:
            start += 25
            counter += 1
        else:
            break

except HTTPError as e:
    print(e)
except URLError as e:
    print('Not found!')

KeyboardInterrupt: 

#### Merge and save results

In [4]:
total_results = int(search_results["search-results"]["opensearch:totalResults"])

IDs = [None] * total_results
titles = [None] * total_results
links = [None] * total_results

counter = 0
while counter <= total_files:
    with open(f"scopus_search_results{counter}.json", "r") as search_results:
        search_results = json.load(search_results)
    for index, entry in enumerate(search_results["search-results"]["entry"]):
        IDs[index+(counter*25)] = entry["eid"]
        titles[index+(counter*25)] = entry["dc:title"]
        for link in entry["link"]:
            if link["@ref"] == "scopus":
                links[index+(counter*25)] = entry["link"][entry["link"].index(link)]["@href"]
    counter += 1

In [6]:
abstracts4 = [None] * total_results

for index, link in enumerate(links):
    abstracts4[index+591] = BeautifulSoup(urllib.request.build_opener(urllib.request.HTTPCookieProcessor(CookieJar())).open(Request(links[index+591], headers = {'User-Agent': 'Mozilla/5.0'})).read(), 'html.parser').select("section[id = abstractSection] > p")[0].replace_with("</p>", "")
    print(index+591)


591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840


ValueError: unknown url type: 'None'

In [7]:
articles_list4 = {'IDs': IDs, 'Title': titles, 'Abstract': abstracts4}
articles_list4 = pd.DataFrame(articles_list4)
articles_list4.to_csv("scopus_articles4.csv")

In [9]:
print(articles_list4)

                     IDs                                              Title  \
0     2-s2.0-85127375931  MR volumetry in detection of brain atrophic ch...   
1     2-s2.0-85124238043  Neuroradiological differentiation of white mat...   
2     2-s2.0-85128516163  Magnetic resonance imaging criteria at onset t...   
3     2-s2.0-85125650520  Ceramide kinase knockout ameliorates multiple ...   
4     2-s2.0-85124555237  Expanding the phenotype of HNRNPU-related neur...   
...                  ...                                                ...   
1509   2-s2.0-0014719498  ACID PROTEINASE ACTIVITY OF WHITE MATTER AND P...   
1510   2-s2.0-0004777895  THE RELATIONSHIP BETWEEN ENZYME ACTIVITY AND N...   
1511  2-s2.0-50549161122  Observations on the presence of lysolecithin i...   
1512                None                                               None   
1513                None                                               None   

                                               Abst